In [34]:
import pandas as pd
from sklearn.neighbors import LocalOutlierFactor
from numpy import quantile
import warnings

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)

In [35]:
df = pd.read_csv(r'..\..\datasets\df_minmax_norm.csv', encoding='latin-1')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2065547 entries, 0 to 2065546
Data columns (total 7 columns):
 #   Column            Dtype  
---  ------            -----  
 0   nf_timestamp      float64
 1   emit_lat          float64
 2   emit_long         float64
 3   prod_ncm          float64
 4   prod_quant        float64
 5   prod_valor_unit   float64
 6   prod_valor_total  float64
dtypes: float64(7)
memory usage: 110.3 MB


In [36]:
model = LocalOutlierFactor(n_neighbors=20) 
model.fit_predict(df)
df['anomaly_score'] = model.negative_outlier_factor_ 

outlier_thresh = 0.0001

thresh = quantile(df['anomaly_score'], outlier_thresh)

print(thresh)

df_outliers = df[df['anomaly_score'] <= thresh]

df_outliers

-19629377.29281153


,nf_timestamp,emit_lat,emit_long,prod_ncm,prod_quant,prod_valor_unit,prod_valor_total,anomaly_score
81721,0.804903,0.722844,0.882210,0.732010,2.500000e-08,7.165586e-06,1.433117e-05,-4.607790e+07
89467,0.811374,0.722844,0.882210,0.731815,3.750000e-08,8.240424e-08,2.472127e-07,-2.236185e+07
92963,0.796039,0.722844,0.882210,0.732310,2.500000e-07,2.078020e-08,4.156040e-07,-2.075610e+07
97045,0.794087,0.722844,0.882210,0.732310,3.750000e-07,2.006364e-08,6.019093e-07,-3.033400e+07
106733,0.801373,0.722844,0.882210,0.731815,1.000000e-07,7.165586e-08,5.732469e-07,-2.154490e+07
...,...,...,...,...,...,...,...,...
2064546,0.460382,0.271683,0.582269,0.901891,1.250000e-08,1.323527e-05,1.323527e-05,-8.936697e+07
2064547,0.460382,0.271683,0.582269,0.940490,2.500000e-08,3.829289e-07,7.657146e-07,-9.977806e+07
2064550,0.460382,0.271683,0.582269,0.902110,1.250000e-08,1.323484e-06,1.323484e-06,-8.930825e+07
2065523,0.759402,0.067002,0.431981,0.901780,6.250000e-08,7.165586e-07,3.582793e-06,-7.679298e+07


In [37]:
df_outliers['idx'] = df_outliers.index
df_outliers = df_outliers.sort_values(by='anomaly_score', ascending=True)
df_outliers.to_csv('output\outliers.csv', index=False)
df_outliers

,nf_timestamp,emit_lat,emit_long,prod_ncm,prod_quant,prod_valor_unit,prod_valor_total,anomaly_score,idx
2054885,0.431162,0.330731,0.811273,0.401693,5.875000e-07,0.000000e+00,0.000000e+00,-7.604445e+08,2054885
2054884,0.431162,0.330731,0.811273,0.401693,5.025000e-06,0.000000e+00,0.000000e+00,-7.604445e+08,2054884
982078,0.377824,0.257826,0.641096,0.481950,1.250000e-07,1.318468e-08,1.318468e-07,-4.783431e+08,982078
982077,0.377824,0.257826,0.641096,0.481950,1.250000e-07,1.318468e-08,1.318468e-07,-4.783431e+08,982077
1089371,0.324483,0.256107,0.710816,0.847190,1.250000e-08,3.008113e-05,3.008113e-05,-4.326628e+08,1089371
...,...,...,...,...,...,...,...,...,...
106733,0.801373,0.722844,0.882210,0.731815,1.000000e-07,7.165586e-08,5.732469e-07,-2.154490e+07,106733
92963,0.796039,0.722844,0.882210,0.732310,2.500000e-07,2.078020e-08,4.156040e-07,-2.075610e+07,92963
2055048,0.141429,0.238141,0.587462,0.901831,1.250000e-03,1.576429e-09,1.526270e-04,-1.974616e+07,2055048
2055056,0.141429,0.238141,0.587462,0.901831,1.250000e-03,1.576429e-09,1.526270e-04,-1.974616e+07,2055056
